In [71]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import Embedding
from keras.layers import Conv1D, LSTM, Conv2D, MaxPooling1D, MaxPooling2D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from sklearn.preprocessing import StandardScaler
from keras.optimizers import Adam, SGD
import pandas as pd
import numpy as np
from tflscripts import concat_and_reindex

In [62]:
dataset_path = '../datasets/scott-final-iter1-fft/'

device = '128.237.248.186'

df = pd.read_pickle(dataset_path + device + '.p')
df_labels = pd.read_pickle(dataset_path + device + '_labels.p')

# device = '128.237.247.134'

# df2 = pd.read_pickle(dataset_path + device + '.p')
# df2_labels = pd.read_pickle(dataset_path + device + '_labels.p')

# device = '128.237.248.186'

# df3 = pd.read_pickle(dataset_path + device + '.p')
# df3_labels = pd.read_pickle(dataset_path + device + '_labels.p')

# df, df_labels = concat_and_reindex([df1, df2], [df1_labels, df2_labels])

# filter activities
use_activities = [0,1,2,3,4,5,7,8,9,20]
num_classes = len(use_activities)
df_labels = df_labels.loc[df_labels.label.isin(use_activities)]

In [63]:
# scaling makes acc worse for some reason...
# scaler = StandardScaler()

# df[df.columns] = scaler.fit_transform(df[df.columns])

**Rename the labels**

In [64]:
for i, activity in enumerate(use_activities):
    df_labels.loc[df_labels.label == activity, 'label'] = i

In [80]:
from keras.preprocessing.sequence import pad_sequences

def extract_window_data(window_id):
    segment = df.loc[df.id == window_id]
    # microphone works much better than accelerometer
    data = segment.filter(regex='MICROPHONE').values.T
    data = pad_sequences(data,
                         maxlen=50,
                         dtype='float64',
                         padding='post',
                         value=0)
    return np.expand_dims(data, axis=2)

x = np.array([extract_window_data(id) for id in df_labels.index])
y = df_labels[['label']].values

In [83]:
x.shape

(709, 128, 50, 1)

In [84]:
train_test_split = np.random.rand(len(x)) < 0.70
x_train = x[train_test_split]
y_train = y[train_test_split]
x_test = x[~train_test_split]
y_test = y[~train_test_split]

y_train = keras.utils.to_categorical(y_train, num_classes=num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes=num_classes)

In [85]:
batch_size = 64
timesteps = 128
data_dim = 50

In [86]:
import math

x_train = x_train[:math.floor(len(x_train) / batch_size) * batch_size]
x_test = x_test[:math.floor(len(x_test) / batch_size) * batch_size]
y_train = y_train[:math.floor(len(y_train) / batch_size) * batch_size]
y_test = y_test[:math.floor(len(y_test) / batch_size) * batch_size]

In [87]:
model = Sequential()
model.add(ConvLSTM2D(filters=40, kernel_size=(3, 3),
                   input_shape=(None, timesteps, data_dim, 1),
                   padding='same', return_sequences=True))
# model.add(LSTM(32, return_sequences=True, stateful=True,
#                batch_input_shape=(batch_size, timesteps, data_dim)))
# model.add(LSTM(32, return_sequences=True, stateful=True))
# model.add(LSTM(32, stateful=True))
model.add(Dropout(1))
model.add(Dense(num_classes, activation='softmax'))

opt = Adam(lr=0.001)
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

model.fit(x_train, y_train,
          validation_data=(x_test, y_test),
          batch_size=batch_size, epochs=50)

score = model.evaluate(x_test, y_test, batch_size=batch_size)
score

ValueError: Error when checking model input: expected conv_lst_m2d_3_input to have 5 dimensions, but got array with shape (448, 128, 50, 1)

In [ ]:
model.evaluate(x_train, y_train, batch_size=batch_size)